TP1 - Algoritmos 2 Geometria computacional

#Triangulação , Corte de orelhas e Coloração de polígonos


#Imports

In [1]:
import fractions
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import os
import ipywidgets as widgets
from IPython.display import display, Image, clear_output
import imageio.v2 as imageio  # Importar a versão 2 explicitamente
import tempfile
import time

#Insira aqui o arquivo que contenha o polígono

In [2]:
caminho_arquivo = input()

# Carregamento e Leitura do arquivo do polígono


A função `Ler_poligono_de_arquivo`  lê as coordenadas dos vértices de um polígono a partir de um arquivo de texto. O arquivo contém uma linha com o número de vértices seguido das coordenadas x e y de cada vértice, separadas por espaços. A função converte as coordenadas de frações para floats e retorna uma lista de tuplas, onde cada tupla representa um vértice do polígono.

In [3]:
def Ler_poligono_de_arquivo(diretorio_arquivo):
    with open(diretorio_arquivo, 'r') as arquivo:
        linha = arquivo.readline().strip().split()
        num_vertices = int(linha[0])
        poligono = [(float(fractions.Fraction(linha[i])), float(fractions.Fraction(linha[i + 1]))) for i in range(1, len(linha), 2)]

    return  poligono

In [4]:
poligono = Ler_poligono_de_arquivo(caminho_arquivo)
poligono_original =  Ler_poligono_de_arquivo(caminho_arquivo)

#Visualização do polígono recebido

In [5]:
x_coords, y_coords = zip(*poligono)

# Crie a figura
fig = go.Figure()

# Adicione o polígono
fig.add_trace(go.Scatter(x=[x for x in x_coords]+[x_coords[0]], y=[y for y in y_coords]+[x_coords[0]], fill='toself'))

# Adicione um título e exiba o gráfico
fig.update_layout(title='Polígono', template='plotly_white', xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
fig.show()

#Implementação da Triangulação pelo algoritmo Ear Clipping


A função E_orelha verifica se um vértice do polígono forma uma "orelha".

Verificação de Ângulo: Calcula o produto vetorial dos vetores formados pelos vértices adjacentes para determinar se o ângulo é convexo (positivo) ou côncavo (negativo).

Verificação de Vértices Internos: Verifica se algum outro vértice do polígono está dentro do triângulo formado pelo vértice atual e seus vizinhos. Se algum vértice estiver dentro do triângulo, o vértice atual não é uma orelha.

Retorno: Se o ângulo é convexo e não há vértices internos, retorna True, indicando que o vértice é uma orelha.

In [6]:
def E_orelha(poligono, i):
    n = len(poligono)
    vertice_anterior = poligono[(i-1) % n]
    vertice_atual = poligono[i]
    vertice_seguinte = poligono[(i+1) % n]
    # Verifica se o ângulo no vértice atual é concâvo
    produto_vetorial = (vertice_atual[0] - vertice_anterior[0]) * (vertice_seguinte[1] - vertice_atual[1]) \
                       - (vertice_atual[1] - vertice_anterior[1]) * (vertice_seguinte[0] - vertice_atual[0])
    if produto_vetorial < 0:
        return False

    # Verifica se não há nenhum vértice dentro do triângulo formado pelo vértice atual e seus vizinhos
    for j in range(n):
        if j != i and j != (i-1) % n and j != (i+1) % n:
            ponto = poligono[j]
            if ponto_esta_dentro_de_triangulo(ponto, vertice_anterior, vertice_atual, vertice_seguinte):
                return False

    return True

Explicação: A função ponto_esta_dentro_de_triangulo verifica se um ponto está dentro de um triângulo formado por três vértices.

Cálculo dos Produtos Vetoriais: Calcula os produtos vetoriais dos vetores formados pelo ponto e os três vértices do triângulo.

Verificação de Lados: Verifica se o ponto está do mesmo lado de cada aresta do triângulo.

Retorno: Se o ponto está do mesmo lado de todas as arestas, retorna True, indicando que o ponto está dentro do triângulo. Caso contrário, retorna False.

In [7]:
def ponto_esta_dentro_de_triangulo(ponto, a, b, c):
    # Verifica se o ponto está do mesmo lado de todos os lados de um triângulo
    ab = (b[0] - a[0]) * (ponto[1] - a[1]) - (b[1] - a[1]) * (ponto[0] - a[0])
    bc = (c[0] - b[0]) * (ponto[1] - b[1]) - (c[1] - b[1]) * (ponto[0] - b[0])
    ca = (a[0] - c[0]) * (ponto[1] - c[1]) - (a[1] - c[1]) * (ponto[0] - c[0])

    return (ab >= 0 and bc >= 0 and ca >= 0) or (ab <= 0 and bc <= 0 and ca <= 0)

In [8]:
def Corte_de_orelhas(poligono):
  n = len(poligono)
  triangulos = []

  while n > 2:
      for i in range(n):
          if E_orelha(poligono, i):
              vertice_anterior = poligono[(i-1) % n]
              vertice_atual = poligono[i]
              vertice_seguinte = poligono[(i+1) % n]

              triangulos.append((vertice_anterior, vertice_atual, vertice_seguinte))
              poligono.remove(vertice_atual)
              n -= 1
              break
  return triangulos

#Criação das imagens do processo do corte de orelhas

Explicação: A função printandoPoligono plota o polígono e opcionalmente destaca uma "orelha" encontrada durante o processo de triangulação.

Coordenadas do Polígono: As coordenadas x e y dos vértices do polígono são extraídas e fechadas para formar um ciclo.

Configuração do Gráfico: Um gráfico é configurado com um tamanho ajustado.
Plotagem do Polígono: O polígono é plotado com bordas azuis e preenchimento semitransparente.

Destacar Orelha: Se uma orelha for fornecida, ela é plotada com bordas laranjas e preenchimento semitransparente.

Informações do Passo: Se informações do passo forem fornecidas, elas são adicionadas como título do gráfico.

Eixos e Legenda: Os eixos são rotulados e uma grade é adicionada. Uma linha horizontal e uma vertical são desenhadas no eixo 0. A legenda é configurada.

Salvar Frame: Se um número de quadro e uma lista de arquivos temporários forem fornecidos, o gráfico é salvo como uma imagem temporária e fechado.

In [9]:
def printandoPoligono(poligono, orelha=None, frame_num=None, temp_files=None, step_info=None):
    x, y = zip(*poligono)
    x += (x[0],)
    y += (y[0],)

    plt.figure(figsize=(8.4, 7), dpi=100)
    plt.plot(x, y, marker='o', color='#1f77b4', linestyle='-', linewidth=2, markersize=5.6, label='Polígono Original')
    plt.fill(x, y, alpha=0.2, color='#1f77b4')

    if orelha:
        ox, oy = zip(*orelha)
        ox += (ox[0],)
        oy += (oy[0],)
        plt.plot(ox, oy, marker='o', color='#ff7f0e', linestyle='-', linewidth=2, markersize=5.6, label='Orelha Encontrada')
        plt.fill(ox, oy, alpha=0.5, color='#ff7f0e')

    if step_info:
        plt.title(step_info, fontsize=11.2, fontweight='bold', color='#2ca02c')

    plt.xlabel('Coordenada X', fontsize=9.8)
    plt.ylabel('Coordenada Y', fontsize=9.8)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.axhline(0, color='black', linewidth=0.5)
    plt.axvline(0, color='black', linewidth=0.5)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.legend(loc='upper right', fontsize=8.4)

    if frame_num is not None and temp_files is not None:
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
        plt.savefig(temp_file.name)
        temp_files.append(temp_file.name)
    plt.close()


A função Corte_de_orelhas_animacao realiza a triangulação de um polígono usando o método de corte de orelhas e cria uma animação do processo.

Inicialização: O número de vértices (n), a lista de triângulos resultantes (triangulos), o número do quadro da animação (frame_num) e os arquivos temporários para os frames da animação (temp_files) são inicializados.

Loop de Triangulação: Enquanto o polígono tiver mais de três vértices, ele continua a procurar e remover "orelhas":

Encontrar Orelha: Para cada vértice, verifica se ele forma uma "orelha" com seus vizinhos.

Plotar Orelha: Se uma orelha é encontrada, ela é destacada no gráfico e salva como um frame na animação.

Remover Orelha: A orelha encontrada é removida da lista de vértices do polígono.
Plotar Polígono Atualizado: O polígono atualizado é plotado e salvo como um frame na animação.

Retorno: A função retorna a lista de triângulos e os arquivos temporários dos frames da animação.

In [10]:
def Corte_de_orelhas_animacao(poligono):
    n = len(poligono)
    triangulos = []
    frame_num = 0
    temp_files = []

    while n > 2:
        for i in range(n):
            if E_orelha(poligono, i):
                orelha = [poligono[(i-1) % n], poligono[i], poligono[(i+1) % n]]
                step_info = f'Encontrada orelha em {poligono[i]} com vértices {orelha}'
                printandoPoligono(poligono, orelha, frame_num, temp_files, step_info)
                frame_num += 1

                vertice_anterior = poligono[(i-1) % n]
                vertice_atual = poligono[i]
                vertice_seguinte = poligono[(i+1) % n]

                triangulos.append((vertice_anterior, vertice_atual, vertice_seguinte))
                poligono.remove(vertice_atual)
                n -= 1

                step_info = f'Removida orelha em {vertice_atual}. Polígono atualizado.'
                printandoPoligono(poligono, frame_num=frame_num, temp_files=temp_files, step_info=step_info)
                frame_num += 1
                break

    return triangulos, temp_files

triangulos, temp_files = Corte_de_orelhas_animacao(poligono)

In [11]:
def criar_gif(temp_files, gif_name='corte_de_orelhas.gif'):
    images = []
    for temp_file in temp_files:
        images.append(imageio.imread(temp_file))
    imageio.mimsave(gif_name, images, duration=0.5)
    print(f'GIF salvo como {gif_name}')

def mostrar_frame(frame_num, temp_files):
    display(Image(filename=temp_files[frame_num]))

def play_gif(slider, temp_files):
    for i in range(slider.max + 1):
        slider.value = i
        time.sleep(0.5)

if not os.path.exists('frames'):
  os.makedirs('frames')

criar_gif(temp_files)

num_frames = len(temp_files)

slider = widgets.IntSlider(min=0, max=num_frames-1, step=1, description='Frame:')
interactive_plot = widgets.interactive(mostrar_frame, frame_num=slider, temp_files=widgets.fixed(temp_files))

play_button = widgets.Button(description="Play GIF")
play_button.on_click(lambda x: play_gif(slider, temp_files))


GIF salvo como corte_de_orelhas.gif


#Visualização das etapas de descoberta e corte de orelhas

In [12]:
display(play_button)
display(interactive_plot)

Button(description='Play GIF', style=ButtonStyle())

interactive(children=(IntSlider(value=0, description='Frame:', max=15), Output()), _dom_classes=('widget-inter…

# Criação do grafo dual

Neste passo realizamos a criação do grafo dual, de modo que cada face se torna um vértice e temos uma aresta entre faces adjacentes, nesse caso em específico, os vértices são os  triângulos e as arestas se eles compartilham uma diagonal. Na animação abaixo pode-se ver o passo a passo da inserção dos vértices e arestas do grafo dual, em que selecionamos o centro do triangulo como a posição do vértice do dual. Além disso,  como as diagonais dividem o polígono em dois, a remoção de qualquer aresta do grafo obtido o desconecta. Portanto, temos uma árvore.

In [13]:
def triangulos_sao_adjacentes(triangulo1, triangulo2):
    vertices_comuns = 0
    for vertice1 in triangulo1:
        for vertice2 in triangulo2:
            if vertice1 == vertice2:
                vertices_comuns += 1
    return vertices_comuns == 2

In [14]:
def Criar_grafo_dual(triangulos):
    grafo_dual = {}
    for i in range(len(triangulos)):
        grafo_dual[i] = []
        for j in range(len(triangulos)):
            if i != j and triangulos_sao_adjacentes(triangulos[i], triangulos[j]):
                grafo_dual[i].append(j)
    return grafo_dual

# Visualização do grafo obtido

In [15]:
# Função para calcular o centro de um triângulo
def centro_triangulo(triangulo):
    x = (triangulo[0][0] + triangulo[1][0] + triangulo[2][0]) / 3
    y = (triangulo[0][1] + triangulo[1][1] + triangulo[2][1]) / 3
    return (x, y)

centros = [centro_triangulo(triangulos[i]) for i in range(len(triangulos))]

def Criar_grafo_dual_animacao(triangulos, centros):
    grafo_dual = {}
    frames = []  # Para armazenar os frames da animação

    # Dados iniciais para o primeiro frame (grafo vazio)
    data = [go.Scatter(
        x=[],
        y=[],
        mode='markers+lines',
        marker=dict(size=10),
        text=[],
        hoverinfo='text',
        line=dict(width=2, color='red')
    )]
    frames.append(go.Frame(data=data, name='Frame 0'))

    for i in range(len(triangulos)):
        grafo_dual[centros[i]] = []

        # Adiciona o novo vértice ao frame
        data[0]['x'] = [i[0] for i in grafo_dual.keys()]
        data[0]['y'] = [i[1] for i in grafo_dual.keys()]
        data[0]['text'] = [f'Triângulo {i}' for i in grafo_dual.keys()]
        frames.append(go.Frame(data=data.copy(), name=f'Frame {i}'))

        for j in range(len(triangulos)):
            if i != j and triangulos_sao_adjacentes(triangulos[i], triangulos[j]):
                grafo_dual[centros[i]].append(centros[j])
                # Adiciona a nova aresta (i, j) ao frame
                frames.append(go.Frame(data=[
                    *data,  # Manter os dados existentes
                    go.Scatter(
                        x=[centros[i][0], centros[j][0]],
                        y=[centros[i][1], centros[j][1]],
                        mode='lines',
                        line=dict(color='blue', width=3)
                    )
                ], name=f'Frame {i}-{j}'))

    return grafo_dual, frames


grafo_dual, frames = Criar_grafo_dual_animacao(triangulos, centros)

valores_x = [v[0] for v in centros]
valores_y = [v[1] for v in centros]
min_x, max_x = min(valores_x), max(valores_x)
min_y, max_y = min(valores_y), max(valores_y)

# Cria o gráfico
fig = go.Figure(
    data=frames[0].data,  # Começa com os dados do primeiro frame
    layout=go.Layout(
        title="Criação do Grafo Dual",
        xaxis=dict(title="Triângulos", range=[min_x-0.25, max_x+0.25]),
        yaxis=dict(range=[-2*min_y, 2*max_y]),
        plot_bgcolor='rgba(0,0,0,0)',
        updatemenus=[
            dict(
                type="buttons",
                buttons=[
                    dict(
                        label="Play",
                        method="animate",
                        args=[None, {"frame": {"duration": 800, "redraw": True},
                                     "fromcurrent": True, "transition": {"duration": 300, "easing": "cubic-in-out"}}]
                    ),
                    dict(  # Botão de Pause
                        label="Pause",
                        method="animate",
                        args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate",
                                       "transition": {"duration": 0}}]
                    )
                ]
            )
        ]
    ),
    frames=frames
)

fig.show()

# Coloração do polígono

A coloração do polígono pode ser feita realizando uma busca em profundidade (DFS) no grafo dual. Assim, a busca se inicia em um vértice arbitrário do dual e colorimos cada vértice do triângulo equivalente com uma cor. Ao visitar o próximo vértice, teremos apenas uma escolha para  a cor do vértice restante do triângulo equivalente, pois os outros dois são compartilhados com o triângulo anterior, obtemos assim uma 3-coloração do polígono.

In [16]:
def colorir_vertices_poligono(grafo, triangulo_atual, triangulo_visitados=None, triangulos_coloridos=None):
    if triangulo_visitados is None:
        triangulo_visitados = set()
    triangulo_visitados.add(triangulo_atual)
    colorir_vertices_triangulo(triangulos_coloridos[triangulo_atual])
    for triangulo_adjacente in grafo[triangulo_atual]:
        if triangulo_adjacente not in triangulo_visitados:
            atualizar_cores_vertices_triangulos_adjacentes(triangulos_coloridos, triangulo_atual, triangulo_adjacente)
            colorir_vertices_poligono(grafo, triangulo_adjacente, triangulo_visitados, triangulos_coloridos)

In [17]:
def atualizar_cores_vertices_triangulos_adjacentes(triangulos, indice1, indice2):
    triangulo1 = triangulos[indice1]
    triangulo2 = triangulos[indice2]
    vertices_comuns = set(tuple(vertice[0:2]) for vertice in triangulo1) \
        & set(tuple(vertice[0:2]) for vertice in triangulo2)
    for vertice in triangulo2:
        if tuple(vertice[0:2]) in vertices_comuns and vertice[2] == "white":
            for v in triangulo1:
                if tuple(v[0:2]) == tuple(vertice[0:2]) and v[2] != "white":
                    vertice[2] = v[2]
                    break

In [18]:
def triangulos_com_vertices_coloridos(triangulos):
    triangulos_coloridos = []
    for triangulo in triangulos:
        triangulo_colorido = []
        for vertice in triangulo:
            vertice_colorido = [*vertice, "white"]
            triangulo_colorido.append(vertice_colorido)
        triangulos_coloridos.append(triangulo_colorido)
    return triangulos_coloridos

In [19]:
def colorir_vertices_triangulo(triangle):
    cores = {"red", "blue", "green"}
    vertices_coloridos = set(vertice[2] for vertice in triangle)
    cor_faltante = cores.difference(vertices_coloridos).pop()
    for vertice in triangle:
        if vertice[2] == "white":
            vertice[2] = cor_faltante
            break

In [20]:
triangulos_com_cores = triangulos_com_vertices_coloridos(triangulos)
triangulos_com_cores[0][0][2] = "red"
triangulos_com_cores[0][1][2] = "blue"
grafo_dual = Criar_grafo_dual(triangulos)

# Visualização do passo a passo da coloração do polígono

In [21]:
def colorir_vertices_poligono(grafo, triangulo_atual, triangulo_visitados=None, triangulos_coloridos=None):
    if triangulo_visitados is None:
        triangulo_visitados = set()
    triangulo_visitados.add(triangulo_atual)

    # Cria frame para animação
    frames.append(criar_frame(triangulos_coloridos))
    time.sleep(1)  # Aguarda 1 segundo para visualização

    colorir_vertices_triangulo(triangulos_coloridos[triangulo_atual])
    for triangulo_adjacente in grafo[triangulo_atual]:
        if triangulo_adjacente not in triangulo_visitados:
            atualizar_cores_vertices_triangulos_adjacentes(triangulos_coloridos, triangulo_atual, triangulo_adjacente)
            colorir_vertices_poligono(grafo, triangulo_adjacente, triangulo_visitados, triangulos_coloridos)
            frames.append(criar_frame(triangulos_coloridos))
            time.sleep(1)  # Aguarda 1 segundo para visualização


# Função para criar um frame da animação
def criar_frame(triangulos_coloridos):
    data = []
    for vertices in triangulos_coloridos:  # Itera diretamente sobre os valores (vértices)
        x = [vertice[0] for vertice in vertices] + [vertices[0][0]]
        y = [vertice[1] for vertice in vertices] + [vertices[0][1]]
        cores = [vertice[2] for vertice in vertices] + [vertices[0][2]]
        data.append(go.Scatter(x=x, y=y, mode='lines+markers',
                                line=dict(color='black'), marker=dict(color=cores, size=10),
                                textposition='top center'))
    return go.Frame(data=data)

# Inicializa lista de frames para a animação
frames = []

# Chama a função principal para colorir o grafo e gerar os frames
colorir_vertices_poligono(grafo_dual, 0, triangulos_coloridos=triangulos_com_cores)

valores_x = [v[0] for v in poligono_original]
valores_y = [v[1] for v in poligono_original]
min_x, max_x = min(valores_x), max(valores_x)
min_y, max_y = min(valores_y), max(valores_y)

# Cria a figura do Plotly com os frames da animação
fig = go.Figure(
    data=frames[0].data,
    layout=go.Layout(
        xaxis=dict(range=[min_x-0.25, max_x+0.25], autorange=False),  # Ajuste os limites do eixo x conforme necessário
        yaxis=dict(range=[min_y - 0.25, max_y + 0.25], autorange=False),  # Ajuste os limites do eixo y conforme necessário
        title="Animação Coloração de Grafo Triangulado",
        plot_bgcolor='rgba(0,0,0,0)',
        updatemenus=[dict(
            type="buttons",
            buttons=[
                dict(label="Play",
                    method="animate",
                    args=[None, {"frame": {"duration": 1000, "redraw": True},
                                    "fromcurrent": True, "transition": {"duration": 500}}]),
                dict(label="Pause",
                    method="animate",
                    args=[None, {"frame": {"duration": 0, "redraw": False},
                                    "mode": "immediate",
                                    "transition": {"duration": 0}}])
            ])]
    ),
    frames=frames
)

fig.show()